In [6]:
# Need to run these from your terminal
# Create python environment
# !python -m venv venv
# install jupyter notebook
# !pip install jupyter

In [8]:
import os
import wget

# Function for creating the necessary paths

In [9]:
def create_paths(custom_model_name):
    """ Method to create the needed paths """
    tf_record_script = 'generate_tfrecord.py'
    label_map = 'label_map.pbtxt'

    # Setting up Folder Structure
    paths = {
        'tensorflow_path': os.path.join('Tensorflow'),
        'workspace_path': os.path.join('Tensorflow', 'workspace'),
        'scripts_path': os.path.join('Tensorflow', 'scripts'),
        'api_path': os.path.join('Tensorflow', 'models'),
        'annotation_path': os.path.join('Tensorflow', 'workspace', 'annotations'),
        'image_path': os.path.join('Tensorflow', 'workspace', 'images'),
        'train_image_path': os.path.join('Tensorflow', 'workspace', 'images', 'train'),
        'test_image_path': os.path.join('Tensorflow', 'workspace', 'images', 'test'),
        'model_path': os.path.join('Tensorflow', 'workspace', 'models'),
        'pretrained_model_path': os.path.join('Tensorflow', 'workspace', 'pre-trained-models'),
        'checkpoint_path': os.path.join('Tensorflow', 'workspace', 'models', custom_model_name),
        'output_path': os.path.join('Tensorflow', 'workspace', 'models', custom_model_name, 'export'),
        'protoc_path': os.path.join('Tensorflow', 'protoc')
    }

    files = {
        'pipeline_config': os.path.join('Tensorflow', 'workspace', 'models', custom_model_name, 'pipeline.config'),
        'tf_record_script': os.path.join(paths['scripts_path'], tf_record_script),
        'labelmap': os.path.join(paths['annotation_path'], label_map)
    }

    # Create Folder Structure
    for path in paths.values():
        if not os.path.exists(path):
            os.mkdir(path)

    return paths, files

# Function for downloading needed files

In [10]:
def download_files(paths):
    if linux:
        protobuff = "https://github.com/protocolbuffers/protobuf/releases/download/v25.3/protoc-25.3-linux-x86_64.zip"
        zip = "protoc-25.3-linux-x86_64.zip"
    elif mac:
        protobuff = "https://github.com/protocolbuffers/protobuf/releases/download/v26.0-rc3/protoc-26.0-rc-3-osx-universal_binary.zip"
        zip = "protoc-26.0-rc-3-osx-universal_binary.zip"
    else:
        protobuff = "https://github.com/protocolbuffers/protobuf/releases/download/v25.3/protoc-25.3-win64.zip"
        zip = "protoc-25.3-win64.zip"
    
    # urls
    download_urls = {
        'protobuff': protobuff,
        'tensorflow_models': "https://github.com/tensorflow/models/archive/refs/heads/master.zip",
        'pretrained_model': "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz"
    }
    
    pretrained_model = 'ssd_mobilenet_v2_320x320_coco17_tpu-8'

    # Download files
    wget.download(download_urls["protobuff"], paths['protoc_path'])
    wget.download(download_urls["tensorflow_models"], paths['api_path'])
    wget.download(download_urls["pretrained_model"], paths['pretrained_model_path'])

    # Extract Files
    if linux or mac:
        !cd {paths['protoc_path']} && unzip {zip}
        !cd {paths['api_path']} && unzip models-master.zip -d ./tmp && mv ./tmp/*/* .
    else:
        !cd {paths['protoc_path']} && tar -xf {zip}
        !cd {paths['api_path']} && tar -xf models-master.zip --strip-components 1
        
    !cd {paths['pretrained_model_path']} && tar -zxvf {pretrained_model + '.tar.gz'}

    # add protoc to path
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['protoc_path'], 'bin'))

# Function for verifying environment

In [11]:
def verify_install(paths, setup=False):
    if setup:
        if linux or mac:
            !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py setup.py && python setup.py build && python setup.py install
        else:
            !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
        !cd Tensorflow/models/research/slim && pip install -e .

    verification_script = os.path.join(paths['api_path'], 'research', 'object_detection', 'builders',
                                       'model_builder_tf2_test.py')
    !python {verification_script}

# Environment Setup

In [12]:
# Change to false if you are on windows
linux=False
mac=False

In [13]:
# Create the paths and get the file and path dictionaries
paths, files = create_paths('hummifier_model')

In [14]:
# Get our execution dependencies
download_files(paths)

Archive:  protoc-26.0-rc-3-osx-universal_binary.zip
   creating: include/
   creating: include/google/
   creating: include/google/protobuf/
  inflating: include/google/protobuf/struct.proto  
  inflating: include/google/protobuf/api.proto  
  inflating: include/google/protobuf/wrappers.proto  
  inflating: include/google/protobuf/timestamp.proto  
  inflating: include/google/protobuf/duration.proto  
  inflating: include/google/protobuf/descriptor.proto  
  inflating: include/google/protobuf/type.proto  
  inflating: include/google/protobuf/source_context.proto  
   creating: include/google/protobuf/compiler/
  inflating: include/google/protobuf/compiler/plugin.proto  
  inflating: include/google/protobuf/field_mask.proto  
  inflating: include/google/protobuf/empty.proto  
  inflating: include/google/protobuf/any.proto  
   creating: bin/
  inflating: bin/protoc              
  inflating: readme.txt              
Archive:  models-master.zip
0429a9339527cfc699ae9bea0093879538e94874
  

In [15]:
# Verify installation
# The final outcome should be similar to 24 tests ran
verify_install(paths, setup=True)

running build
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/exporter_lib_tf2_test.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib.py -> build/lib/object_detection
copying object_detection/exporter_tf1_test.py -> build/lib/object_detection
copying object_detection/model_lib.py -> build/lib/object_detection
copying object_detection/exporter.py -> build/lib/object_detection
copying object_detection/exporter_lib_v2.py -> build/lib/object_detection
copying object_detection/export_tflite_graph_lib_tf2_test.py -> build/lib/object_detection
copying object_detection/__init__.py -> build/lib/object_detection
copying object_detection/export_tflite_graph_lib_tf2.py -> build/lib/object_detection
copying object_detection/eval_util_test.py -> build/lib/object_detection
copying object_detection/exporter_main_v2.py -> build/lib/object_detection
copying object_detection/eval_util.py -> build/lib/objec

If you get error: module 'keras._tf_keras.keras.layers' has no attribute 'experimental'

reinstall tensorflow with: pip install tensorflow=2.13.0